# Projeto 2 - Ciência dos Dados

Nome: Arthur Carvalho

Nome: Guilherme Lunetta

___
# Classificador automático de sentimento


## Preparando o ambiente no jupyter:

In [30]:
%%capture

#Instalando o tweepy
!pip install tweepy
!pip install emoji
!pip install emoji_list

In [31]:
import tweepy
import math
import os.path
import pandas as pd
import json
from random import shuffle
import emoji
import emoji_list

___
## Autenticando no  Twitter

* Conta: @tuca1209

In [32]:
#Dados de autenticação do twitter:

#Coloque aqui o identificador da conta no twitter: Arthur FC

#leitura do arquivo no formato JSON
with open('auth.pass') as fp:    
    data = json.load(fp)

#Configurando a biblioteca. Não modificar
auth = tweepy.OAuthHandler(data['consumer_key'], data['consumer_secret'])
auth.set_access_token(data['access_token'], data['access_token_secret'])

___
## Etapas do projeto:

### Escolha de um produto e coleta das mensagens


In [33]:
#Produto escolhido:
produto = 'Trump'

#Quantidade mínima de mensagens capturadas:
n = 500
#Quantidade mínima de mensagens para a base de treinamento:
t = 300

#Filtro de língua, escolha uma na tabela ISO 639-1.
lang = 'pt'

Capturando os dados do twitter:

In [34]:
#Cria um objeto para a captura
api = tweepy.API(auth)

#Inicia a captura, para mais detalhes: ver a documentação do tweepy
i = 1
msgs = []


for msg in tweepy.Cursor(api.search, q=produto, lang=lang, tweet_mode="extended").items():
    if msg.full_text.lower()[0] != 'r' and msg.full_text.lower()[1] != 't':
        msgs.append(msg.full_text.lower())
        i += 1
        if i > n:
            break

            
#Embaralhando as mensagens para reduzir um possível viés
shuffle(msgs)

TweepError: Twitter error response: status code = 429

Salvando os dados em uma planilha Excel:

In [ ]:
#Verifica se o arquivo não existe para não substituir um conjunto pronto
if not os.path.isfile('./{0}.xlsx'.format(produto)):
    
    #Abre o arquivo para escrita
    writer = pd.ExcelWriter('{0}.xlsx'.format(produto))

    #divide o conjunto de mensagens em duas planilhas
    dft = pd.DataFrame({'Treinamento' : pd.Series(msgs[:t])})
    dft.to_excel(excel_writer = writer, sheet_name = 'Treinamento', index = False)

    dfc = pd.DataFrame({'Teste' : pd.Series(msgs[t:])})
    dfc.to_excel(excel_writer = writer, sheet_name = 'Teste', index = False)

    #fecha o arquivo
    writer.save()

___
### Classificando as mensagens na coragem

Esta etapa é manual. Faça a mesma pelo Excel.

___
### Montando o Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [55]:
import re

dados = pd.read_excel('Trump.xlsx')
tweets = dados.iloc[0:,0]

def limpar_caracteres(tabela, titulo):
    a = tabela[titulo]
    a = a.str.lower()

    itens = ['.', ':', ';', '"', "'", '?', '(', ')', '[',']',',', '\n', '\t']
    i2 = ['?', '!']
    emojis = emoji_list.all_emoji
    for emoji in emojis:
        try:
            a = a.str.replace(emoji," {} ".format(emoji) )
        except:
            pass
    for e in itens:
        a = a.str.replace(e,'')
    for e in i2:
        a = a.str.replace(e,' {} '.format(e))

    a = a.str.replace('é','e')
    a = a.str.replace('ê','e')
    a = a.str.replace('á','a')
    a = a.str.replace('ã','a')
    a = a.str.replace('ô','o')
    a = a.str.replace('ó','o')
    a = a.str.replace('ú','u')
    a = a.str.replace('ç','c')
    a = a.str.replace('í','i')
    a = a.str.replace('@',' @')
    a = a.str.replace(',','.')
        
    tabela[titulo] = a

    return tabela

titulo = "Treinamento"
tabela = limpar_caracteres(dados, titulo)

def split_tweet(tabela, titulo):
    a = tabela[titulo].str.split(' ')

    nova = pd.DataFrame()
    nova['palavras'] = []

    for lista in a:
        for e in ['#','' , ' ', 'rt']:
            while lista.count(e) != 0:
                lista.remove(e)

        for palavra in lista:
            if '@' in palavra and palavra != '@realDonaldTrump':
                lista.remove(palavra) 
            if palavra[:4] == 'http':
                lista.remove(palavra)
        nova = nova.append({'palavras': lista}, ignore_index=True)
                
    return nova

nova = split_tweet(dados, titulo)


def tabela_palavras(tabela):
    a = tabela['palavras']
    prob = pd.DataFrame()
    prob['palavras'] = []
    for lista in a:
        for e in lista:
            prob = prob.append({'palavras': e}, ignore_index=True)
    
    return prob['palavras']

def tabela_tudo(tabela1, tabela2):
    a = tabela1['palavras']
    b = tabela2['palavras']
    tudo = pd.DataFrame()
    tudo['palavras'] = []
    for lista in a:
        for e in lista:
            tudo = tudo.append({'palavras': e}, ignore_index=True)
    for lista in b:
        for e in lista:
            tudo = tudo.append({'palavras': e}, ignore_index=True)
    
    return tudo

In [56]:
relevante = dados.loc[(dados['Relevancia'])==1]
irrelevante = dados.loc[(dados['Relevancia'])==0]
relevante

In [57]:
trump_teste = pd.read_excel('TrumpTeste.xlsx')

In [58]:
palavras_rel = tabela_palavras(split_tweet(limpar_caracteres(relevante, 'Treinamento'), 'Treinamento'))
num_rel = palavras_rel.value_counts()
len_rel = len(num_rel)

palavras_irrel = tabela_palavras(split_tweet(limpar_caracteres(irrelevante, 'Treinamento'), 'Treinamento'))
num_irrel = palavras_irrel.value_counts()
len_irrel = len(num_irrel)

C:\Users\Usuario\Anaconda3\lib\site-packages\ipykernel_launcher.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [59]:
teste = split_tweet(limpar_caracteres(trump_teste, 'Teste'), 'Teste')
teste = teste.join(trump_teste['Relevancia'])

tudo = tabela_tudo(split_tweet(limpar_caracteres(relevante, 'Treinamento'), 'Treinamento'),split_tweet(limpar_caracteres(irrelevante, 'Treinamento'), 'Treinamento'))
#Total de palavras diferentes que aparecem na base de treinamento para o LaPlace.
total_palavras = len(tudo['palavras'].value_counts())

C:\Users\Usuario\Anaconda3\lib\site-packages\ipykernel_launcher.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


___
### Verificando a performance

Agora você deve testar o seu classificador com a base de Testes.

In [83]:
i = 0
teste["Avaliacao"] = pd.Series()
teste["Resultado"] = pd.Series()

for linha in teste["palavras"]:
    p_um = 1
    p_zero = 1
    p_naive = ''
    for palavra in linha:
        try:
            #laplace
            p_um *= (num_rel[palavra] + 1)/(len_rel + total_palavras)
            p_zero *= (num_irrel[palavra] + 1)/(len_irrel + total_palavras)
            print(p_um - p_zero)
        except:
            p_um *= 1/(len_rel + total_palavras)
            p_zero *= 1/(len_irrel + total_palavras)
    if p_um > p_zero:
        p_naive = '1'
        teste["Avaliacao"][i] = "1"
        if teste['Relevancia'][i] == p_naive:
            teste["Resultado"][i] = "Positivo Verdadeiro"
        elif teste['Relevancia'][i] != p_naive:
            teste["Resultado"][i] = "Positivo Falso"

    elif p_um < p_zero:
        p_naive = '0'
        teste["Avaliacao"][i] = "0"
        if teste['Relevancia'][i] == p_naive:
            teste["Resultado"][i] = "Negativo Verdadeiro"
        elif teste['Relevancia'][i] != p_naive:
            teste["Resultado"][i] = "Negativo Falso"
    i+=1

-0.004845936913390057
-1.8852396854332025e-08
-9.66319304510497e-10
-3.2682348197760693e-12
-3.939744187457557e-14
-2.63259057338063e-17
-8.553975397798755e-23
-3.310619033889279e-28
-1.3653919548294772e-29
-2.654851822782744e-35
-6.083013657893117e-38
-7.43341329698792e-40
-0.00044245893400530003
-1.3173016413878627e-06
-7.059261201344295e-09
-5.2195141209025065e-12
-1.3180699664352936e-17
-5.838089421286315e-21
-4.246047352237006e-24
-3.129088511978535e-29
-3.281079888053177e-31
-3.086177460492569e-36
-1.557928711874121e-39
-1.4130855494282963e-44
-1.2530838016030204e-54
-9.570351953273214e-58
-3.947076418704984e-59
-9.209559224624428e-65
-1.1937284431353319e-71
-9.421193336772792e-74
-6.9555609845861e-76
-1.3975133590639656e-81
-1.603248728328571e-86
-1.6833443349214356e-91
-2.571299894187435e-94
-8.332585576757162e-101
-7.506744362975791e-106
-4.913942189903783e-111
-1.910901647825324e-117
-3.715494576861854e-124
-1.1558857519314557e-129
-5.296828805569941e-132
-0.00985739275868601

C:\Users\Usuario\Anaconda3\lib\site-packages\ipykernel_launcher.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\Usuario\Anaconda3\lib\site-packages\ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


-0.00040910591628289927
-9.796345045405206e-11
1.331385977311308e-16
-4.202468967063992e-18
-3.1450560069180152e-24
-5.038387001910556e-30
-1.3062807497775765e-35
-5.886466099703387e-37
-2.111136066824471e-05
1.1988484025618584e-06
5.803027333817361e-14
-1.0535840559271449e-14
-3.313148996740625e-20
-1.7223908405765222e-25
-8.208789635897661e-27
-2.961951276241226e-28
-2.233265969472238e-31
-4.988135129047962e-38
-6.942158189424802e-41
-8.002653492182826e-54
-6.04353512026895e-56
-7.05535331369698e-62
6.670603544480175e-05
-2.9680864750362156e-10
-1.9432865073922604e-15
-1.4835761475373746e-18
-7.698979863459299e-25
-3.175241367677388e-26
-2.0579452873648337e-32
-2.619584146665178e-35
-6.002102249012611e-37
-9.168181030256587e-40
-3.5010874412959625e-42
-8.395793019064169e-48
-7.61810230842998e-53
-3.141885396036788e-54
-4.7992139294982297e-57
-4.323564639159996e-62
-1.948245787715506e-63
-6.9438495577080304e-65
-3.88912143551321e-67
-2.545833054789854e-72
-2.3100160556462252e-77
-3.52

2.638529739785237e-07
-1.3482227570366259e-09
-2.2422274746933452e-18
-2.0499686219537638e-25
-3.8080747651895604e-30
-9.944313107327086e-36
-3.4560697690901594e-44
-3.519547907307252e-46
-3.2257272454223625e-48
-2.9269322898650826e-53
-5.691048105467212e-60
-1.3039605807827229e-61
-9.95889276154237e-65
-2.788918870761646e-67
-2.530584996077112e-72
-8.955028281484404e-74
-7.374005120256457e-90
-3.94231435739106e-92
-1.4051018588717424e-93
-1.430857291262739e-96
-2.3184267113287907e-102
7.89476924989577e-05
2.322677577601531e-07
-8.3781427968417e-19
-5.489086878623279e-24
-1.720834925015046e-33
-8.922497801513004e-40
-3.180116324462474e-41
-3.916098878706594e-47
-5.076222928998916e-54
-3.769109570715967e-63
-1.554469833136589e-64
-4.3531487180510983e-67
-1.9615766881239757e-68
-3.9950645379310295e-71
-5.085367283517262e-74
-5.178581755109209e-77
-4.6653333316926465e-82
-1.0689409364876257e-83
-2.993470035988767e-86
-9.894688652065433e-92
-1.038901060246997e-96
-0.003620744961275315
-0.0

C:\Users\Usuario\Anaconda3\lib\site-packages\ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\Usuario\Anaconda3\lib\site-packages\ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


...

-0.014827872039351576
-9.490767981842766e-08
-4.602924774419287e-13
-2.5484158665586993e-15
-3.593658656834812e-22
-1.8046153909166388e-27
-3.011154930928603e-33
1.298699871069122e-11
1.2180001547806988e-13
-5.436492898518757e-19
-8.70042119807312e-25
-5.68486653887257e-27
-3.0313501159289137e-28
-7.122130018938142e-31
-6.701295694765855e-41
-1.5355357830100987e-43
-3.909041372542747e-46
-1.8910589095287795e-48
-1.7036617080260517e-53
-1.76670155106758e-59
-9.445235166376286e-62
-0.003968642592287429
-3.615673502021521e-05
-2.1852116049042125e-10
-1.0510397413967508e-11
-8.626421302604695e-14
-3.11819329273783e-15
-3.726361165482244e-21
-2.3780607327952276e-33
-9.807684363389046e-35
-4.244669970621445e-37
-1.9126949789506428e-38
-1.9477545634011446e-41
-3.7871501689221457e-48
-8.748349567218773e-57
-1.3041003338035774e-62
-4.226083290612058e-69
-5.164256566939377e-71
-2.629458537138174e-74
-1.3388281757322671e-77
-1.4622185025264522e-90
-0.014827872039351576
-5.533094251558428e-06
-4.3

-8.783747457428738e-15
-1.7289001588257295e-17
3.494025013015767e-24
1.5576676574371067e-29
-1.0084020121923673e-36
8.802685658861602e-59
-7.88629819144498e-66
-2.607386732140493e-71
-7.30209364154421e-74
-4.259122263552102e-80
-9.758319295606519e-83
-1.024628566575131e-87
-1.5650583926705414e-90
-2.3240645453740485e-99
-5.976576246091391e-101
-3.04189222065588e-104
-8.873812993988015e-110
-3.660095879221094e-111
-2.0498458993231924e-113
-7.306240666131877e-115
-3.01348071098846e-116
-1.3580087926941551e-117
-0.002709217982422447
-2.543509112078599e-05
-3.3287511927428957e-07
-3.419521529807026e-09
-2.521217457604219e-11
-1.8870216121994099e-13
-1.40940573836517e-15
-7.664052596297476e-18
-1.1627647604532218e-20
-2.0168319040303864e-23
-3.198226295686582e-26
-7.383295470548685e-28
-3.074133738416909e-29
-1.3989711782619223e-30
-4.997262117019813e-32
-2.5021358532062402e-37
-5.013771119882469e-40
-0.009588442820172842
-1.434140923561854e-07
-7.770663894585032e-09
-2.728005793689157e-11


-3.5370492369025395e-06
-1.629453038063655e-11
-8.106566203583646e-14
-2.3337909416765273e-22
-1.2060431196223019e-23
-4.423471585283812e-25
-1.2399519877530264e-30
-2.2535489513503374e-36
-4.1789121048772295e-42
-1.9867612633996402e-48
-0.003968642592287429
-0.0001630506081807235
-9.663212791309545e-10
-6.064334237499122e-12
-6.52091612196876e-17
-3.041644652947212e-19
-1.3989623763301195e-20
-1.1618201640957355e-22
-2.4303468456336713e-28
-0.0001545234315578482
-8.935593578127282e-06
-3.0379701426995155e-07
-6.026355856562524e-10
-1.0848990720834472e-12
-1.7492447029045446e-19
-6.053083580624664e-25
-2.6529805665480176e-26
-1.4349608921200469e-28
-5.245421246330113e-30
-2.2663918987979117e-31
-1.0699144742456856e-32
-6.971893265361756e-35
-7.180451794402111e-37
-4.212028152964948e-43
-5.500079473320972e-50
-2.8004448262555864e-53
-3.5647388876177805e-56
-1.0587357792630685e-65
-2.470285406975094e-71
-9.37476805001136e-81
-2.916478707913507e-86
-3.563925100324929e-88
-4.35510195424396

1.371906896923237e-07
1.8043542397643615e-10
9.868014205999048e-17
-7.940013238720604e-27
-7.0003996565143565e-28
-6.297347614319636e-33
-8.25802095418842e-46
-2.9900727381486866e-48
-6.284343344563767e-51
-3.27239626337147e-53
-1.692880304633301e-59
-2.071286187041363e-76
-5.370020755017261e-82
-3.5443023079468986e-92
-1.11114824609452e-101
-2.566744170120261e-110
-2.3124070233139775e-115
-1.0420080711865044e-116
-2.3875019194266325e-118
-1.14241719592913e-127
-8.303889253759874e-07
-6.208425799086063e-09
-7.484184179269225e-12
-8.08031256573463e-15
-3.5479729700986086e-16
-5.0820634854754016e-21
-3.347272828380853e-33
-1.956418526261862e-39
-6.545158800743324e-45
-6.478237665796174e-48
-8.906299261986349e-51
-9.686208246312597e-53
-5.4528196079961425e-59
-1.4658763142353097e-60
-1.416644936474541e-63
-5.263688337332247e-66
-3.184611957758655e-78
-8.887626587590069e-81
-8.901721331335175e-84
-2.2147764316836864e-90
-2.19853340941221e-93
-1.558340233915373e-95
-4.1191890125173375e-102


-0.004845936913390057
-3.563394662450189e-09
-8.164824791228408e-11
-7.483615301619042e-13
-7.049285421915392e-15
-5.383954656918952e-18
-8.025679298337291e-24
-1.872614674230764e-29
-1.966178308543522e-34
-8.85991614269999e-36
-3.4453639149624074e-42
-6.699115691639897e-48
-2.1709449068188432e-54
-5.628173162960811e-61
-1.0029852880156538e-63
-1.3001125588879814e-70
-1.2871250072938294e-80
-2.9491153414794627e-83
-9.245511079503709e-93
-2.377282635004773e-94
-2.1416705268772703e-99
-8.723708867117189e-102
-7.859103453100886e-107
-5.402133228964459e-109
-6.126933101173733e-122
-1.1515907771534662e-127
-7.622545877288728e-130
-4.0751900056788005e-132
-4.309870707971788e-07
-1.4259317148262742e-12
-1.292330628514654e-17
-1.171238535773994e-22
-6.81312391424439e-29
-2.8771695627427595e-30
-1.3889893367431368e-33
-3.249505246680771e-35
-2.7197052877895523e-45
-2.882663961359131e-47
-3.729603672936479e-50
-9.711795286719022e-52
-4.8721900027734514e-55
-1.440350638295154e-60
-5.9721906193376

In [75]:
teste.head(10)

,palavras,Relevancia,Avaliacao,Resultado
0,"[se, preocupe, com, trump, macron, !, sua, apr...",0,0.0,Negativo Falso
1,"[quem, ainda, tem, alguma, esperanca, da, gran...",0,0.0,Negativo Falso
2,"[o, filme, e, sobre, o, trump]",0,0.0,Negativo Falso
3,"[vai, deixar, o, patrao, trump, bravinho, hein]",1,0.0,Negativo Falso
4,"[ninguem, liga, pra, vcs, mais, nao, perdendo,...",0,0.0,Negativo Falso
5,"[gente, esta, charge, do, bozo, lambendo, as, ...",0,0.0,Negativo Falso
6,"[boris, johnson, ja, mudou, a, sua, posicao, s...",1,0.0,Negativo Falso
7,"[placa, com, o, nome, de, trump, !, ta, doido,...",1,0.0,Negativo Falso
8,"[donald, trump, ataca, john, legend, e, chriss...",1,0.0,Negativo Falso
9,"[obrigado, pt, por, ter, feito, do, brasil, um...",0,0.0,Negativo Falso


___
### Concluindo

## Aperfeiçoamento:

Os trabalhos vão evoluir em conceito dependendo da quantidade de itens avançados:

* Limpar: \n, :, ", ', (, ), etc SEM remover emojis
* Corrigir separação de espaços entre palavras e emojis ou emojis e emojis
* Propor outras limpezas e transformações que não afetem a qualidade da informação ou classificação
* Criar categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante (3 categorias: C, mais categorias conta para B)
* Explicar por que não posso usar o próprio classificador para gerar mais amostras de treinamento
* Propor diferentes cenários para Naïve Bayes fora do contexto do projeto
* Sugerir e explicar melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* Montar um dashboard que periodicamente realiza análise de sentimento e visualiza estes dados

# Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**